In [ ]:
pip install git+https://github.com/openai/whisper.git

In [ ]:
!pip install fairseq


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import whisper

model = whisper.load_model("large-v3")

ModuleNotFoundError: No module named 'whisper'

In [ ]:
model.device

device(type='cpu')

In [ ]:
from IPython.display import Audio
import scipy.io.wavfile as wav

Audio('/content/drive/MyDrive/Colab_Notebooks/T5/input/CUSTOM_DATASET/Speaker1/01_01_01_01_01.wav')


In [ ]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("/content/drive/MyDrive/Colab_Notebooks/T5/input/CUSTOM_DATASET/Speaker1/01_01_01_01_01.wav")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio=audio, n_mels=128)


# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions(fp16 = False)

result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: en
We need an ambulance as soon as possible.


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("badalsahani/text-classification-multi", use_auth_token=True)

tokenizer = AutoTokenizer.from_pretrained("badalsahani/text-classification-multi", use_auth_token=True)

inputs = tokenizer("I love AutoTrain", return_tensors="pt")

outputs = model(**inputs)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


LocalTokenNotFoundError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

In [ ]:
data = '/content/drive/MyDrive/Colab_Notebooks/T5/input/CUSTOM_DATASET'

# Now use os.listdir to read the contents of the directory
dir_list = os.listdir(data)
dir_list.sort()

emotion = []
path = []
for i in dir_list:
    # Make sure to join the directory path and the subdirectory/file name
    sub_dir_path = os.path.join(data, i)
    # Check if it's a directory before attempting to list its contents
    if os.path.isdir(sub_dir_path):
        fname = os.listdir(sub_dir_path)
        for f in fname:
            # Skip system files or any files that don't match your expected format
            if f.endswith('.wav'):  # Assuming your files are .wav audio files
                part = f.split('.')[0].split('_')
                emotion.append(int(part[0]))
                file_path = os.path.join(sub_dir_path, f)
                path.append(file_path)

In [ ]:
dir_list = os.listdir(data)
dir_list.sort()

emotion = []
path = []
for i in dir_list:
    fname = os.listdir(data + i)
    for f in fname:
        part = f.split('.')[0].split('_')
        emotion.append(int(part[0]))
        path.append(data + i + '/' + f)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab_Notebooks/T5/input/CUSTOM_DATASETSpeaker1'

In [ ]:
path

[]

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="michellejieli/emotion_text_classifier")
classifier("We need an ambulance as soon as possible.!")

[{'label': 'neutral', 'score': 0.47950103878974915}]

In [ ]:
import pandas as pd
import numpy as np

import os
import sys
import tensorflow as tf

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
import IPython.display as ipd
from IPython.display import Audio

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM,BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore"
                         )
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="michellejieli/emotion_text_classifier")

result = classifier("We need an ambulance as soon as possible.!", label=["stressful", "painful", "angry"])

print(result)

TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'label'

In [ ]:
!pip install transformers[torch] accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers.optimization import AdamW

# Custom labels
custom_labels = ["angry", "painful",  "drunk" , "stressful" ]

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
model = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")

# Prepare training data
train_texts= result.text
#train_texts = ["i feel so angry","We need an ambulance as soon as possible.!" , "we hhhh  whhf deas sick ",  "i feel so nervious"]
train_labels = ["angry", "painful",  "drunk" , "stressful" ]

train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="pt")

train_labels_encoded = [custom_labels.index(label) for label in train_labels]
train_labels_tensor = torch.tensor(train_labels_encoded)

train_dataset = TensorDataset(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
model.train()
for epoch in range():
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Average Loss: {avg_loss}")



AssertionError: Size mismatch between tensors

In [ ]:
print(train_encodings["input_ids"].size())
print(train_encodings["attention_mask"].size())
print(train_labels_tensor.size())

torch.Size([1, 11])
torch.Size([1, 11])
torch.Size([4])


In [ ]:
# Save the trained model
model.save_pretrained("./custom_sentiment_model")

In [ ]:
# Load the trained model
custom_sentiment_classifier = AutoModelForSequenceClassification.from_pretrained("./custom_sentiment_model")

In [ ]:
def predict_emo(input_text):

    input_encodings = tokenizer(input_text, truncation=True, padding=True, return_tensors="pt")
    input_ids = input_encodings["input_ids"]
    attention_mask = input_encodings["attention_mask"]

    output = custom_sentiment_classifier(input_ids=input_ids, attention_mask=attention_mask)
    logits = output.logits
    predicted_label_idx = torch.argmax(logits, dim=1).item()
    predicted_label = custom_labels[predicted_label_idx]

    print(f"Predicted Label: {predicted_label}")
    return predicted_label


In [ ]:
y_pred = []
for t in train_texts:
    predicted_label = predict_emo(t)
    y_pred.append(predicted_label)

print(y_pred)

Predicted Label: angry
Predicted Label: angry
Predicted Label: drunk
Predicted Label: stressful
['angry', 'angry', 'drunk', 'stressful']


In [ ]:
accuracy_score(train_labels ,y_pred)

0.75

In [ ]:
train_texts

['i feel so angry',
 'We need an ambulance as soon as possible.!',
 'we hhhh  whhf deas sick ',
 'i feel so nervious']

In [ ]:
train_labels

['angry', 'painful', 'drunk', 'stressful']